In [199]:
import torch
import pandas as pd
model = torch.load("zhu_imp.pth")

In [202]:
NUM_OF_JUNCTIONS = 8
ROLLING_WINDOW_LENGTH = 300
TRAFFIC_DATASET = "/home/local/ASURITE/speddira/dev/archived/traffic_sense_net/city_scale/raw_datasets/trafficDataset_2024-2-16_1915hours_2592000steps.csv"
SIM_DURATION = 2_592_000
generate_incidents_csv = False

In [203]:
dtype = {
    'step' : 'int64',
    'time_of_day' : 'int64',
    'identified_edge' : 'object',
    'junction_mean_speed' : 'float64',
    'traffic_count' : 'float64',
    'traffic_occupancy' : 'float64',
    'vehicles_per_lane_1' : 'int64',
    'vehicles_per_lane_0' : 'int64',
    'lane_mean_speed_0' : 'float64',
    'lane_mean_speed_1' : 'float64',
    'incident_edge': 'object',
    'incident_start_time': 'float64',
    'incident_type': 'object',
    'accident_label': 'bool',
    'accident_id': 'object',
    'accident_duration': 'float64',
    'incident_lane': 'object'
}
df_traffic = pd.read_csv(TRAFFIC_DATASET,dtype=dtype)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_data = df_traffic[['step','identified_edge','traffic_count','accident_label']]

In [ ]:
df_incident = df_data[["step","accident_label"]]

In [ ]:
sensors_combinations = {
    "senor_1_2":["533573776#0","436794680#0"],
    "sensor_1_4":["5607328#0","436791113#0"],
    "sensor_2_1":["-1088637809#1","436794670"],
    "sensor_2_3":["436794679#0","-436794679#3"],
    "sensor_2_5":["436942385#0","1051038541#0"],
    "sensor_3_2":["-436794676#1","436794669"],
    "sensor_3_6":["531969915#0","436942357"],
    # "sensor_4_1":["",""],
    # "sensor_4_5":["",""],
    # "sensor_4_7":["",""],
    "sensor_5_2":["436942381#0","-436942381#3"],
    "sensor_5_4":["436790495","533371302#0"],
    "sensor_5_6":["30031286#0","436790491"],
    "sensor_5_8":["-1033824750","436942374"],
    "sensor_6_3":["-436942362#3","436942362#0"],
    "sensor_6_5":["436789580#1","436789564#0"],
    "sensor_6_9":["-436942356#1","436942356#0"],
    "sensor_7_4":["519448767","436940278"],
    "sensor_7_8":["436940270","-643913497"],
    "sensor_8_5":["436943774","-613687451#1"],
    "sensor_8_7":["436943745#0","-436943745#2"],
    "sensor_8_9":["436943742","351673438"],
    "sensor_9_6":["-436943762#2","436943762#0"],
    "sensor_9_8":["436943750#0","436943743#0"]        
}


In [ ]:
pivot_df = df_data.pivot(index='step', columns='identified_edge', values='traffic_count')

In [ ]:
pivot_df.reset_index(inplace=True)

In [ ]:
pivot_df.head()

In [ ]:
pivot_df.shape

In [ ]:
for i in sensors_combinations:
    
    
    value = sensors_combinations[i]
    pivot_df[i] = pivot_df[value[0]]+pivot_df[value[1]]
    
    pivot_df = pivot_df.drop([value[0],value[1]],axis=1)

In [ ]:
for key in sensors_combinations:
    
    column = f"{key}_rolling"
    pivot_df[key] = pivot_df[key].rolling(window=300).sum()